In [1]:
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from tqdm.notebook import tqdm
import time
import os
import copy
from PIL import Image
import glob 
import syft as sy

In [2]:
n_clients = 4
validation_set_size = 0.2
min_imgs_per_client = 300
federated = True
batch_size = 4
data_dir = './UCMerced_LandUse/Images'
multilabel_excel_file = './multilabels/LandUse_Multilabeled.xlsx'
class_names =  np.array(["airplane","bare-soil","buildings","cars","chaparral","court","dock","field","grass","mobile-home","pavement","sand","sea","ship","tanks","trees","water"])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
import pandas as pd

df = pd.read_excel (multilabel_excel_file)
df_label = np.array(df)

if federated:
    hook = sy.TorchHook(torch)
    hosts=[]
    for i in range(n_clients):
        hosts.append(sy.VirtualWorker(hook, id="host_"+str(i)))
    print(hosts)

[<VirtualWorker id:host_0 #objects:0>, <VirtualWorker id:host_1 #objects:0>, <VirtualWorker id:host_2 #objects:0>, <VirtualWorker id:host_3 #objects:0>]


In [4]:
largestxor = 0 
largestij = (0,0)

for i in range(1,17):
    for j in range(i+1,18):
        #colnand = np.sum(np.logical_not(np.logical_and(df_label[:,i], df_label[:,j])))
        colxor = np.sum(np.logical_xor(df_label[:,i].astype(bool) , df_label[:,j].astype(bool) )) -  np.sum(np.logical_and(df_label[:,i], df_label[:,j]))
        #print(i,j, colxor, colnand)
        if colxor >= largestxor and np.sum(df_label[:,i]) >=700 and np.sum(df_label[:,j])>= 700 :
            largestxor = colxor
            largestij = (i,j)
print(largestxor,class_names[largestij[0]-1], class_names[largestij[1]-1], largestij)

674 bare-soil cars (2, 4)


In [5]:
def uncor_selecter(nr_label = 4,min_img = 300):
    """retrun a list with the least correlated labels """
    image_perlabel = np.sum(df_label[:,1:],axis= 0)
    biggest_label =np.where(np.any([image_perlabel > min_img],axis=0))[0]
    #print(biggest_label, image_perlabel[biggest_label])

    selected_list = [] 
    allcor_lost = np.array([0,0,0])
    for i in range(0,len(biggest_label)-1):
        it = biggest_label[i]
        for j in range(i+1,len(biggest_label)):
            jt = biggest_label[j]

            colxor = np.sum(np.logical_xor(df_label[:,it].astype(bool) , df_label[:,jt].astype(bool) )) -  np.sum(np.logical_and(df_label[:,it], df_label[:,jt]))
            allcor_lost = np.vstack((allcor_lost, np.array([colxor,it,jt]))) 
    sorted_list = allcor_lost[allcor_lost[:,0].argsort()]
    selected_list.append(sorted_list[-1,1])
    selected_list.append(sorted_list[-1,2])
    #print(sorted_list, selected_list)        

    while len(selected_list)<nr_label:
        biggest_label = np.setdiff1d(biggest_label,np.array(selected_list))
        largestxor = 0 
        largestind = 0
        for i in biggest_label:
            overall_xor = 0 
            for j in (selected_list):
                overall_xor += np.sum(np.logical_xor(df_label[:,i].astype(bool) , df_label[:,j].astype(bool) )) -  np.sum(np.logical_and(df_label[:,i], df_label[:,j]))

            if overall_xor >= largestxor:
                largestxor = overall_xor
                largestind = i

        selected_list.append(largestind)
    
    return selected_list


trylist = uncor_selecter()
print(type(trylist), trylist)

<class 'list'> [1, 2, 3, 8]


In [6]:
import random
def sampler_split_for_client(cdata, subset, nr_client= 4, minimum_skew_percentage = .4):
    selected_labels = uncor_selecter(nr_client, min_imgs_per_client)
    
    splitlists = []
    for sb in selected_labels:
        splitlists.append([])
    for i, (data, label) in enumerate(subset):
        nplabel = label.numpy()
        if np.any(nplabel[selected_labels] == 1):
            if random.random() < minimum_skew_percentage:
                
                flip = np.random.randint(np.sum(nplabel[selected_labels] == 1)) 
                mask = np.where(nplabel[selected_labels] == 1)[0][flip]
                splitlists[mask].append(i)
            
            else:
                flip = np.random.randint(nr_client) 
                splitlists[flip].append(i)
                    
        else:
            flip = np.random.randint(nr_client) 
            splitlists[flip].append(i)

    
    for alist in splitlists:
        print(len(alist))
    return splitlists
    


In [7]:
from torch.utils.data import Dataset
from natsort import natsorted
# Built to handle multilabel dataset vs standard ImageFolder dataset from torchvision
class CustomDataSet(Dataset): 
    def __init__(self, main_dir, transform, labelmat):
        self.main_dir = main_dir
        self.transforms = transform
        self.all_imgs = glob.glob(os.path.join(main_dir, '**/*.tif'), recursive=True)
        self.total_imgs = natsorted(self.all_imgs)
        self.xlabels = labelmat
        
    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = self.total_imgs[idx]
        imagebaselabel = os.path.splitext(os.path.basename(img_loc))[0]
        label = self.xlabels[np.where(self.xlabels[:,0] == imagebaselabel),1:].reshape(17).astype(np.int64)
        tensor_label =  torch.from_numpy(label)
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transforms(image)
        return tensor_image, tensor_label

In [8]:
class DatasetFromSubset(Dataset):
    """
    Helper to convert subsets to datasets since PySyft only works with the datasets
    """

    def __init__(self, subset):
        data, targets = self.subset_to_dataset(subset)
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        return self.data[index, :], self.targets[index]

    @staticmethod
    def subset_to_dataset(subset):
        """
        Method to turn the index tensor and original dataset in subsets into smaller datasets
        :param subset: Subset to transform
        :return: dataset
        """
        indices = subset.indices
        targets = subset.dataset.dataset.xlabels

        if isinstance(indices, list):
            pass
        elif isinstance(indices, torch.tensor):
            indices = list(indices.data.numpy())
        elif isinstance(indices, np.ndarray):
            indices = list(indices)

        else:
            print(type(indices))
            raise NotImplementedError

        dataloader = torch.utils.data.DataLoader(subset, batch_size=batch_size, shuffle=False)

        for ii, (data, target) in enumerate(dataloader):
            #del target
            if ii == 0:
                concat_tensor = data
                targets_subset = target
            else:
                concat_tensor = torch.cat((concat_tensor, data), 0)
                targets_subset = torch.cat((targets_subset, target), 0)

        return concat_tensor, targets_subset


In [9]:
def load_split_train_test(datadir, labelmat,valid_size = .2):
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    loaded_dataset = CustomDataSet(datadir, transform=train_transforms, labelmat=labelmat)
    #loaded_dataset = datasets.ImageFolder(data_dir)
    total_dataset_size = len(loaded_dataset)
    split = int(np.floor(valid_size * total_dataset_size))
    valid_set = torch.utils.data.Subset(loaded_dataset, range(split))  # take first 10%
    train_set = torch.utils.data.Subset(loaded_dataset, range(split, total_dataset_size))  # take the rest  
    if federated:
        train_set_splits = sampler_split_for_client(loaded_dataset, train_set, n_clients)
        train_set = [DatasetFromSubset(torch.utils.data.Subset(train_set,split)) for split in train_set_splits]
    
    return train_set, valid_set

In [10]:
train_set, valid_set = load_split_train_test(data_dir, df_label, validation_set_size)

if federated:
    assert len(train_set) == len(hosts), "The number of clients and data partitions do not match."
    federated_train_loader = []
    for i in range(n_clients):
        # Create pointers
        data_pointer = train_set[i].data.send(hosts[i])
        target_pointer = train_set[i].data.send(hosts[i])
        federated_train_loader.append((data_pointer, target_pointer))
    dataloaders = {"train" : federated_train_loader, "val" : valid_set}
else:
    dataloaders = {"train" : train_set, "val" : valid_set}
print(dataloaders)

413
387
410
470
{'train': [((Wrapper)>[PointerTensor | me:6798300316 -> host_0:7663311820], (Wrapper)>[PointerTensor | me:60483217454 -> host_0:7663311820]), ((Wrapper)>[PointerTensor | me:49810964230 -> host_1:24968077953], (Wrapper)>[PointerTensor | me:30009381488 -> host_1:24968077953]), ((Wrapper)>[PointerTensor | me:56675311591 -> host_2:31804325160], (Wrapper)>[PointerTensor | me:63363394836 -> host_2:31804325160]), ((Wrapper)>[PointerTensor | me:43133442417 -> host_3:76440359498], (Wrapper)>[PointerTensor | me:52379361958 -> host_3:76440359498])], 'val': <torch.utils.data.dataset.Subset object at 0x7fbd6c647910>}


In [11]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        super(CNN, self).__init__()
        from collections import OrderedDict
        self.conv1 = nn.Conv2d(3, 16, kernel_size=(5, 5))
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(5, 5))
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(5, 5))
        #self.conv4 = nn.Conv2d(64, 128, kernel_size=(5, 5))
        self.linear1 = nn.Linear(64 * 24 * 24, 120)
        self.linear2 = nn.Linear(120, 84)
        self.linear3 = nn.Linear(84, n_classes)                                
    def forward(self, x):
        """
        Args:
          x of shape (batch_size, 1, 28, 28): Input images.
        
        Returns:
          y of shape (batch_size, 10): Outputs of the network.
        """
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.conv3(x)), kernel_size=2, stride=2)
        #x = F.max_pool2d(F.relu(self.conv4(x)), kernel_size=2, stride=2)
        x = x.view(-1, 64 * 24 * 24)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [12]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=5, batch_size=4):
    %matplotlib inline
    import pylab as pl
    from IPython import display
    def live_plot(data):
        pl.plot(data)
        display.clear_output(wait=True)
        display.display(pl.gcf())
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    loss_values, loss_values_val, accuracy_values_val = [], [], []

    for epoch in tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        model.train()  # Set model to training mode
        losses_clients = []

        # Iterate over data.
        for i, (inputs, labels) in enumerate(tqdm(dataloaders['train'])):
            if federated:
                model.send(inputs.location)
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            outputs = model(inputs)
            #_, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if federated:
                # get model (with gradients)
                model.get()
                loss.get()
                
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    output.item(),
                ))
            losses_clients.append(output.item())
        epoch_loss_train = sum(losses_clients)/n_clients
        loss_values.append(epoch_loss_train)
        live_plot(loss_values)
        print('Train Loss: {:.4f} Train Acc: {:.4f}'.format(epoch_loss_train))

        model.eval()   # Set model to evaluate mode
        for i, (inputs, labels) in enumerate(tqdm(dataloaders['val'])):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss_val += loss.get()
            running_corrects_val += torch.sum(preds == labels.data).get()
          
        epoch_loss_val = running_loss_val / len(dataloaders['val'])
        epoch_acc_val = running_corrects_val / len(dataloaders['val'])
        loss_values_val.append(epoch_loss_val)
        accuracy_values_val.append(epoch_acc_val)
        print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(epoch_loss_val, epoch_acc_val))

        # deep copy the model
        if epoch_acc_val > best_acc:
            best_acc = epoch_acc_val
            best_model_wts = copy.deepcopy(model.state_dict())


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    #print(loss_values, accuracy_values, loss_values_val, accuracy_values_val)
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, {"tloss":loss_values, "valloss":loss_values_val, "tacc":accuracy_values,"valacc":accuracy_values_val}

In [13]:
from torchvision import models
from torch.optim import lr_scheduler
import time
import os
import copy

model = CNN(len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.01)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model, stats_dict = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, dataloaders, num_epochs=25, batch_size=batch_size)
torch.save(model.state_dict(), "resnet_fl2.pt")

Epoch 0/24
----------


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target' in call to _thnn_nll_loss_forward